In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)

fish-gcp005
/home/akarshkumar0101/nca-alife
/home/akarshkumar0101/.virtualenvs/nca-alife-torch/bin/python3


In [2]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [3]:
# import experiment_utils
import time
import subprocess

# Launch Experiment (on this node)

In [4]:
header = '\n'.join(['#!/bin/bash', 'source ~/.virtualenvs/nca-alife-torch/bin/activate', 'cd ~/nca-alife/src'])
gpus = [4, 5, 6, 7]
log_dir = os.path.expanduser(f'~/nca-alife-data/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}')
exp_sh = f'./src/experiments/main.sh'
with open(exp_sh, 'r') as f:
    commands = [c for c in f.read().split('\n') if c]
print(f"{exp_sh} has {len(commands)} commands: ")
print('\n'.join(commands[:3]), '\n...')
print(f"Logging to {log_dir}")

./src/experiments/main.sh has 64 commands: 
python clip_img_optim_torch.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/clip_img_optim/prompt=0_aug=0_openai_clip-vit-base-patch32_0"  --prompt="a red apple"                                                --aug_crop=False --aug_perspective=False --aug_color=False --model="openai/clip-vit-base-patch32"  --lr=0.001 --n_iters=10000
python clip_img_optim_torch.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/clip_img_optim/prompt=1_aug=0_openai_clip-vit-base-patch32_0"  --prompt="a photo of a red apple"                                     --aug_crop=False --aug_perspective=False --aug_color=False --model="openai/clip-vit-base-patch32"  --lr=0.001 --n_iters=10000
python clip_img_optim_torch.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/clip_img_optim/prompt=2_aug=0_openai_clip-vit-base-patch32_0"  --prompt="a green tree"                                               --aug_crop=False --aug_perspective=Fal

In [5]:
log_dir

'/home/akarshkumar0101/nca-alife-data/logs/2024_06_11_17_44_54'

In [6]:
!mkdir -p {log_dir}
!rm {os.path.dirname(log_dir)+'/latest'}
!ln -sf {log_dir} {os.path.dirname(log_dir)+'/latest'}
for i_gpu, gpu in enumerate(gpus):
    job_ids = np.arange(len(commands))[i_gpu::len(gpus)]
    cvd = f'export CUDA_VISIBLE_DEVICES={gpu}'
    txt = f"{header}\n{cvd}\n\n"
    for job_id in job_ids:
        txt += f"touch {log_dir}/job_{job_id:05d}.status\n"
        txt += f"{commands[job_id]} &> {log_dir}/job_{job_id:05d}.out\n"
        txt += f"echo $? > {log_dir}/job_{job_id:05d}.status\n\n"
    with open(f"{log_dir}/gpu_{gpu}.sh", "w") as f:
        f.write(txt)

In [7]:
print('\n'.join(txt.split('\n')[:12]))

#!/bin/bash
source ~/.virtualenvs/nca-alife-torch/bin/activate
cd ~/nca-alife/src
export CUDA_VISIBLE_DEVICES=7

touch /home/akarshkumar0101/nca-alife-data/logs/2024_06_11_17_44_54/job_00003.status
python clip_img_optim_torch.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/clip_img_optim/prompt=3_aug=0_openai_clip-vit-base-patch32_0"  --prompt="a very fat cat"                                             --aug_crop=False --aug_perspective=False --aug_color=False --model="openai/clip-vit-base-patch32"  --lr=0.001 --n_iters=10000 &> /home/akarshkumar0101/nca-alife-data/logs/2024_06_11_17_44_54/job_00003.out
echo $? > /home/akarshkumar0101/nca-alife-data/logs/2024_06_11_17_44_54/job_00003.status

touch /home/akarshkumar0101/nca-alife-data/logs/2024_06_11_17_44_54/job_00007.status
python clip_img_optim_torch.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/clip_img_optim/prompt=7_aug=0_openai_clip-vit-base-patch32_0"  --prompt="a photo of the emergence of open-ended

In [9]:
for i_gpu, gpu in enumerate(gpus):
    command = f"bash {log_dir}/gpu_{gpu}.sh"
    print(command)
    process = subprocess.Popen(command, shell=True)

bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_11_17_44_54/gpu_4.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_11_17_44_54/gpu_5.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_11_17_44_54/gpu_6.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_11_17_44_54/gpu_7.sh


# Track Experiment

In [65]:
a = []
for job_id in range(len(commands)):
    a.append(os.path.exists(f'{log_dir}/job_{job_id:05d}.out'))

In [66]:
np.array(a).mean()

1.0

In [67]:
len(commands)

64